In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy import signal
import random

import h5py
from IPython.utils import io

import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy import signal
import random
import scipy.stats as stats

import h5py
from IPython.utils import io

from joblib import Parallel, delayed
from tqdm import tqdm
import pandas as pd

from mpl_toolkits.axes_grid1 import make_axes_locatable
import copy 

import scipy.interpolate as interpolate
import scipy.constants as constants

In [2]:
import sys
sys.path.append('../../../../nadavp/new_trap/Tools/')
sys.path.append('../../../../nadavp/new_trap/Tools/StatFramework/')
import BeadDataFile
from discharge_tools import *

from likelihood_calculator import likelihood_analyser
from likelihood_calculator import GravityFramework
from likelihood_calculator import auxiliary_functions as aux
gfw = GravityFramework.GravityFramework()

sys.path.append('/home/analysis_user/New_trap_code/SensitivityFramework/')
from signal_model_utilities_v3b import *

In [3]:
%matplotlib widget
%reload_ext autoreload
%autoreload 2

### Setup parameters

In [4]:
df = pd.read_pickle("/data/new_trap_processed/processed_files/20200320/Bead1/Shaking/Shaking378/Shaking3_aux.pkl") #here one could put the respective file name
height_cal = df["z_distance_emp"]
time = df['Time_Epoch']
time -= time[0]

In [5]:
fname = r'/home/data_transfer/data_from_windows/20200320/Bead1/TransFunc/Repeat_TransFunc_20200104/TransFunc_X_m300k_250s_1hz.h5'
tf_x = BeadDataFile.BeadDataFile(fname)
fname = r'/home/data_transfer/data_from_windows/20200320/Bead1/TransFunc/Repeat_TransFunc_20200104/TransFunc_Y_m300k_250s_1hz.h5'
tf_y = BeadDataFile.BeadDataFile(fname)
fname = r'/home/data_transfer/data_from_windows/20200320/Bead1/TransFunc/Repeat_TransFunc_20200104/TransFunc_Z_m300k_250s_1hz.h5'
tf_z = BeadDataFile.BeadDataFile(fname)
gfw.build_transfer_function([tf_x, tf_y, tf_z], base_freq=1, number_of_harmonics=90, scale_freq=71, plot=False)

No height data
No cantilever data
Error loading spinning data
No laser power data
No height data
No cantilever data
Error loading spinning data
No laser power data
No height data
No cantilever data
Error loading spinning data
No laser power data


In [282]:
bdf_discharge = load_dir(dirname='/data/new_trap/20200320/Bead1/Discharge/Discharge_after_Mass_20200402/After_spinning_again_20200403/',
                         start_file=230, max_file=40)
bdf_z = bdf_discharge[5:35]

530  files in folder
40  files loaded


In [284]:
bandwidth = 2
m1_zscale = gfw.build_z_response(bdf_z, drive_freq=71, charges=6, bandwidth=bandwidth, decimate=10)
gfw.scale_Z2 *= -1
gfw.scale_Z2

***************************************************
Z2-amplitude:  2.70e+02
reduced chi2:  175.24060921459508
***************************************************
Z2-amplitude:  2.61e+02
reduced chi2:  392.683293718069
***************************************************
Z2-amplitude:  2.75e+02
reduced chi2:  233.0708308046279
***************************************************
Z2-amplitude:  2.79e+02
reduced chi2:  341.81841601739336
***************************************************
Z2-amplitude:  2.81e+02
reduced chi2:  320.1151243975466
***************************************************
Z2-amplitude:  2.75e+02
reduced chi2:  235.238026011364
***************************************************
Z2-amplitude:  2.70e+02
reduced chi2:  250.1490478568309
***************************************************
Z2-amplitude:  2.66e+02
reduced chi2:  270.7275591713835
***************************************************
Z2-amplitude:  2.67e+02
reduced chi2:  318.6342737392779
*******************

1.8474326786101574e+17

In [8]:
def get_Yoffset(i):
    fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking378/Shaking3_'+str(i)+'.h5'
    bb = BeadDataFile.BeadDataFile(fname=fname)    
    return (437.3 - 25 * 9.5) - (aux.voltage_to_position(np.mean(bb.cant_pos[1])))
def get_Stroke(i):
    fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking378/Shaking3_'+str(i)+'.h5'
    bb = BeadDataFile.BeadDataFile(fname=fname)    
    return -np.std(aux.voltage_to_position(bb.cant_pos[1]))*np.sqrt(2)*2
def get_Dist(i):
    fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking378/Shaking3_'+str(i)+'.h5'
    bb = BeadDataFile.BeadDataFile(fname=fname)    
    return 392.2 - aux.voltage_to_position(np.mean(bb.cant_pos[0]))

In [9]:
z_sep = np.mean(height_cal)
y_offset = np.mean(np.array(Parallel(n_jobs=10)(delayed(get_Yoffset)(i) for i in tqdm(range(0, 10000, 100)))))
stroke = np.mean(np.array(Parallel(n_jobs=10)(delayed(get_Stroke)(i) for i in tqdm(range(0, 10000, 100)))))
dist = np.mean(np.array(Parallel(n_jobs=10)(delayed(get_Dist)(i) for i in tqdm(range(0, 10000, 100)))))
G_lambda = 10
print(z_sep, y_offset, stroke, dist, G_lambda)

100%|██████████| 100/100 [00:00<00:00, 696.54it/s]


-15.859100393 4.88214369679 -202.765748131 13.9887662292 10


In [10]:
def func_filter(i):
    fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking378/Shaking3_'+str(i)+'.h5'
#     fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking2_20200403/NoShaking_1_'+str(i)+'.h5'
#     fname = r'/data/new_trap/20201211/Bead1/InitialTest/Data'+str(18+i)+'.h5'
    bb = BeadDataFile.BeadDataFile(fname=fname)
    res = bb.z2
    return (res)

In [11]:
test_phases = np.array(Parallel(n_jobs=46)(delayed(func_filter)(i) for i in tqdm(range(10000))))

100%|██████████| 10000/10000 [01:49<00:00, 91.48it/s]


In [12]:
def func_sideband(x, gravity_freqs_all):
    res = []
    for freq_ in gravity_freqs_all:
        b, a = signal.butter(3, [2.*(freq_+1.5-bandwidth/2.)/5000, 2.*(freq_+1.5+bandwidth/2.)/5000], btype = 'bandpass')
        responsefilt = signal.filtfilt(b, a, x)
        res.append(np.std(responsefilt))
    return np.array(res)

In [296]:
ll = likelihood_analyser.LikelihoodAnalyser()

def get_pl(i):

    data_z2 = np.mean(test_phases[i*100:(i+1)*100], axis=0)
    
    fit_kwargs = {'A': 0, 'phi': 0, 'sigma':13.85,
              'error_A': 1, 'error_phi': 0.5, 'errordef': 1, 'error_sigma': 10,
              'limit_phi': [-1*phi_lim/2, phi_lim/2],
              'limit_A': [-100000, 100000], 'limit_sigma': [0, 1000000],
              'print_level': 0, 'fix_A': False, 'fix_phi': True, 'fix_sigma': False}
    
    noises_std = np.array([1])
#     noises_std = func_sideband(data_z2, gravity_freqs_all)
#     noises_std = noises_std/np.min(noises_std)

    mm_tmp = ll.find_mle_multiHarmoincs2(x=data_z2, template=shaking378_template, scales=gravity_scales, phases=gravity_phases, 
                                          signal_freqs=gravity_freqs_all, bandwidth=bandwidth, noises=noises_std,
                                          decimate=10, **fit_kwargs)
    print('MLEs: ', mm_tmp.values)
    
    PL_array = 2*10*ll.get_PL_multiHarmonics(A_array=A_array, **fit_kwargs)
#     PL_array -= np.min(np.interp(np.arange(np.min(A_array),np.max(A_array),0.0001), A_array, PL_array))
        
    return PL_array

In [102]:
from signal_model_utilities_v3d import force_vs_time as force_vs_time3d

In [281]:
gfw.scale_Z2

-1.4335500432103729e+18

In [304]:
# likelihood calculator stuff
gravity_freqs_all = np.array([36, 33, 12, 6, 21, 18, 39])
G_lambda = 10

test_shaking378 = force_vs_time(dist,z_sep,stroke,3,G_lambda,"z", 
                                yuk_or_grav="yuk",offset_y=y_offset, alpha=1e8, bead_size=7.6)
shaking378_template = np.array(test_shaking378[1])
A_array = np.arange(-5,5,0.1)

fft_angles = np.angle(np.fft.rfft(shaking378_template))
freq1 = np.fft.rfftfreq(len(shaking378_template), d=1./5000)
freq_resp = 3*np.arange(1,14)
fft_angle_resp = np.array([1.378,1.018,0.765,0.657,0.455,0.344,0.513,0.330,0.380,0.626,0.681,0.620,0.425])
fft_angle_sigma = np.array([0.195,0.060,0.115,0.103,0.108,0.094,0.090,0.029,0.253,0.118,0.127,0.133,0.156])

PLs = []
for drive_freq in gravity_freqs_all:
    phase_th = fft_angles[freq1==drive_freq]  ## theory 
    
    phase_th2 = fft_angle_resp[freq_resp==drive_freq] ## response phase
    phase_th2 -= fft_angle_sigma[freq_resp==drive_freq] ## response phase - one sigma
    phase_all = (phase_th+(phase_th2)+np.pi/2)%(2*np.pi)    
    gravity_phases = np.array([phase_all])
    gravity_freqs = np.array([drive_freq])
    gravity_scales = np.array([gfw.scale_Z2])* np.interp(gravity_freqs, gfw.tf_freq, gfw.tf_ffts[2])
    print('\n********************************************')
    print(gravity_freqs, gravity_phases, gravity_scales)
    print('********************************************\n')
    PLs.append(Parallel(n_jobs=46)(delayed(get_pl)(i) for i in tqdm(range(100))))

Loading Gravity Data... 



  0%|          | 0/100 [00:00<?, ?it/s]

Done!
81 9.74070978211e-06
Loaded Yukawa Force

********************************************
[36] [[ 2.05779633]] [  1.40895736e+17]
********************************************





 92%|█████████▏| 92/100 [00:05<00:00, 18.14it/s]

100%|██████████| 100/100 [00:05<00:00, 19.69it/s]

  0%|          | 0/100 [00:00<?, ?it/s]


********************************************
[33] [[ 0.554]] [  1.19510640e+17]
********************************************



...



 92%|█████████▏| 92/100 [00:05<00:00, 18.20it/s]

100%|██████████| 100/100 [00:05<00:00, 19.76it/s]

  0%|          | 0/100 [00:00<?, ?it/s]


********************************************
[18] [[ 4.96238898]] [  1.01891987e+17]
********************************************





 92%|█████████▏| 92/100 [00:05<00:00, 18.03it/s]

100%|██████████| 100/100 [00:05<00:00, 19.57it/s]

  0%|          | 0/100 [00:00<?, ?it/s]


********************************************
[39] [[ 0.269]] [  1.27593426e+17]
********************************************





 92%|█████████▏| 92/100 [00:04<00:00, 19.42it/s]

100%|██████████| 100/100 [00:04<00:00, 21.08it/s]

### Make the construction

In [305]:
PLscopy = copy.deepcopy(PLs)
PLs_all = []
for PLs_ in PLscopy:
    PLs5_tmp = PLs_[0]
    for pl in PLs_[1:]:
        PLs5_tmp += pl
    PLs5_tmp -= np.min(np.interp(np.arange(np.min(A_array),np.max(A_array),0.0001), A_array, PLs5_tmp))
    PLs_all.append(PLs5_tmp)

In [306]:
fig ,ax = plt.subplots(1,4,figsize=(9,2.5), sharey=True)
_ = [a.set(ylim=(0,6), xlim=(np.min(A_array),np.max(A_array)), xlabel=r'$\alpha_{fit} \, [10^{8}]$') for a in ax[1:]]
_ = ax[0].set(ylim=(0,6), xlim=(np.min(A_array),np.max(A_array)), xlabel=r'$\alpha_{fit} \, [10^{8}]$', ylabel = 'Profile-Likelihood')
for i,harmonics in enumerate([0 ,1, 2, 3]):
    ax[i].plot(A_array, PLs_all[harmonics], label=str(gravity_freqs_all[harmonics])+' Hz.-shaking')
    ax[i].plot(A_array, A_array*0 + 2.71, 'r--')
_ = [a.legend() for a in ax]
_ = fig.suptitle(r'PL vs. $\alpha_{fit} \, [10^{8}]$', fontsize=10)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [307]:
all_mles = []
for i,pl in enumerate(PLs_all):
    min_alpha = A_array[np.argmin(pl)]
    all_mles.append(min_alpha)
    print(gravity_freqs_all[i], min_alpha, end='  ***  ')
all_mles = np.array(all_mles)

36 1.0  ***  33 1.2  ***  12 1.2  ***  6 -0.4  ***  21 -1.2  ***  18 -1.7  ***  39 0.9  ***  

In [308]:
## cnostruction for each harmonic separately 
PLs_conservative = []
PLscopy = copy.deepcopy(PLs_all)
for pl in PLscopy:
    min_alpha = A_array[np.argmin(pl)]
    min_value = pl[np.argmin(pl)]
    if (min_alpha>(np.min(A_array)-0.2))&(min_alpha<(np.max(A_array)+0.2)):
        if (min_alpha>-0.001):
            for i,A_ in enumerate(A_array):
                if A_<min_alpha:
                    pl[i] = min_value
            PLs_conservative.append(pl)
        else:
            for i,A_ in enumerate(A_array):
                if A_>min_alpha:
                    pl[i] = min_value
            PLs_conservative.append(pl)

In [309]:
## sum all harmonics after construction
PLs_summed_harmonics22 = []
PLscopy = copy.deepcopy(PLs_conservative)
PLs_summed_harmonics22 = PLscopy[0]
for pp_ in PLscopy[1:]:
    PLs_summed_harmonics22 += pp_

In [314]:
_,ax = plt.subplots()
q_alpha = 11.7
PLs_summed_harmonics -= np.min(np.interp(np.arange(np.min(A_array),np.max(A_array),0.001), A_array, PLs_summed_harmonics))
ax.plot(A_array, PLs_summed_harmonics, '*-', label='regular analysis')
ax.plot(A_array, PLs_summed_harmonics6, '*-', label=r'plus 1$\mu$m Z')
ax.plot(A_array, PLs_summed_harmonics7, '*-', label=r'minus 1$\mu$m Z')
ax.plot(A_array, PLs_summed_harmonics8, '*-', label=r'plus 1$\mu$m Y')
ax.plot(A_array, PLs_summed_harmonics9, '*-', label=r'minus 1$\mu$m Y')
ax.legend()
ax.plot(A_array, A_array*0 + 2.71, 'r--')
ax.set(ylim=(0,10), xlim=(np.min(A_array),np.max(A_array)), xlabel=r'$\alpha \, [10^{8}]$', ylabel = 'Profile-Likelihood')

print(np.interp(q_alpha, PLs_summed_harmonics[50:], A_array[50:]))
print(np.interp(q_alpha, PLs_summed_harmonics6[50:], A_array[50:]))
print(np.interp(q_alpha, PLs_summed_harmonics7[50:], A_array[50:]))
delta = np.abs(np.interp(q_alpha, PLs_summed_harmonics7[50:], A_array[50:])-np.interp(q_alpha, PLs_summed_harmonics6[50:], A_array[50:]))/2
delta2 = np.abs(np.interp(q_alpha, PLs_summed_harmonics9[50:], A_array[50:])-np.interp(q_alpha, PLs_summed_harmonics8[50:], A_array[50:]))/2
print(0.9*delta/np.interp(q_alpha, PLs_summed_harmonics[50:], A_array[50:]))
print(0.2*delta2/np.interp(q_alpha, PLs_summed_harmonics[50:], A_array[50:]))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

1.21304969769
1.24813632438
1.41857263317
0.0632260484479
0.0225941302455


In [221]:
_,ax = plt.subplots()
q_alpha = 11.7
ax.plot(A_array, PLs_summed_harmonics, '*-', label='regular analysis')
ax.plot(A_array, PLs_summed_harmonics4, '*-', label=r'plus 1$\mu$m X')
ax.plot(A_array, PLs_summed_harmonics5, '*-', label=r'minus 1$\mu$m X')
ax.plot(A_array, PLs_summed_harmonics13, '*-', label=r'minus 1.5$\mu$m X')
ax.legend()
ax.plot(A_array, A_array*0 + 2.71, 'r--')
ax.set(ylim=(0,10), xlim=(np.min(A_array),np.max(A_array)), xlabel=r'$\alpha \, [10^{8}]$', ylabel = 'Profile-Likelihood')

print(np.interp(q_alpha, PLs_summed_harmonics[50:], A_array[50:]))
print(np.interp(q_alpha, PLs_summed_harmonics4[50:], A_array[50:]))
print(np.interp(q_alpha, PLs_summed_harmonics5[50:], A_array[50:]))
delta = np.abs(np.interp(q_alpha, PLs_summed_harmonics4[50:], A_array[50:])-np.interp(q_alpha, PLs_summed_harmonics5[50:], A_array[50:]))/2
delta = np.abs(np.interp(q_alpha, PLs_summed_harmonics13[50:], A_array[50:])-np.interp(q_alpha, PLs_summed_harmonics[50:], A_array[50:]))
print(delta/np.interp(q_alpha, PLs_summed_harmonics[50:], A_array[50:]))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

1.21304969769
1.58613578904
1.111280229
0.308445198204


In [311]:
_,ax = plt.subplots()
q_alpha = 11.7
PLs_summed_harmonics -= np.min(np.interp(np.arange(np.min(A_array),np.max(A_array),0.001), A_array, PLs_summed_harmonics))
PLs_summed_harmonics2 -= np.min(np.interp(np.arange(np.min(A_array),np.max(A_array),0.001), A_array, PLs_summed_harmonics2))
ax.plot(A_array, PLs_summed_harmonics, '*-', label='regular analysis')
ax.plot(A_array, PLs_summed_harmonics2, '*-', label='phase uncertainty')
ax.plot(A_array, PLs_summed_harmonics22, '*-', label='phase uncertainty')
# ax.plot(A_array, PLs_summed_harmonics3, '*-', label=r'8$\mu$m attractor')
ax.legend()
ax.plot(A_array, A_array*0 + 2.71, 'r--')
ax.set(ylim=(0,10), xlim=(np.min(A_array),np.max(A_array)), xlabel=r'$\alpha \, [10^{8}]$', ylabel = 'Profile-Likelihood')

print(np.interp(q_alpha, PLs_summed_harmonics[50:], A_array[50:]))
# print(np.interp(q_alpha, PLs_summed_harmonics2[50:], A_array[50:]))
print(np.interp(q_alpha, PLs_summed_harmonics3[50:], A_array[50:]))
delta = np.abs(np.interp(q_alpha, PLs_summed_harmonics22[50:], A_array[50:])-np.interp(q_alpha, PLs_summed_harmonics[50:], A_array[50:]))
print(delta/np.interp(q_alpha, PLs_summed_harmonics[50:], A_array[50:]))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

1.21304969769
1.64776746914
0.123756354058


In [246]:
_,ax = plt.subplots()
q_alpha = 11.7
ax.plot(A_array, PLs_summed_harmonics, '*-', label='regular analysis')
ax.plot(A_array, PLs_summed_harmonics10, '*-', label=r'plus 10% response')
ax.plot(A_array, PLs_summed_harmonics11, '*-', label=r'minus 10% response')
ax.plot(A_array, PLs_summed_harmonics12, '*-', label=r'minus 50% response')
ax.legend()
ax.plot(A_array, A_array*0 + 2.71, 'r--')
ax.set(ylim=(0,10), xlim=(np.min(A_array),np.max(A_array)), xlabel=r'$\alpha \, [10^{8}]$', ylabel = 'Profile-Likelihood')

print(np.interp(q_alpha, PLs_summed_harmonics[50:], A_array[50:]))
print(np.interp(q_alpha, PLs_summed_harmonics10[50:], A_array[50:]))
delta = np.abs(np.interp(q_alpha, PLs_summed_harmonics11[50:], A_array[50:])-np.interp(q_alpha, PLs_summed_harmonics10[50:], A_array[50:]))/2
print(delta/np.interp(q_alpha, PLs_summed_harmonics[50:], A_array[50:]))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

1.21304969769
1.10567112154
0.0934230898205


### Resoponse

In [222]:
bdf_response = load_dir(dirname='/data/new_trap/20200320/Bead1/Discharge/Long_Response/',
                         start_file=0, max_file=1000)

No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data


No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data


No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data


In [236]:
m1_zscale = gfw.build_z_response(bdf_response, drive_freq=71, charges=1, bandwidth=bandwidth, decimate=10)

***************************************************
Z2-amplitude:  3.63e+02
reduced chi2:  323.8735571630487
***************************************************
Z2-amplitude:  3.70e+02
reduced chi2:  224.12754270243073
***************************************************
Z2-amplitude:  3.50e+02
reduced chi2:  396.40468798042076
***************************************************
Z2-amplitude:  3.60e+02
reduced chi2:  423.98952361916787
***************************************************
Z2-amplitude:  3.63e+02
reduced chi2:  381.713026429433
***************************************************
Z2-amplitude:  3.56e+02
reduced chi2:  294.7272921146942
***************************************************
Z2-amplitude:  3.62e+02
reduced chi2:  383.97456086058247
***************************************************
Z2-amplitude:  3.56e+02
reduced chi2:  314.6653028822396
***************************************************
Z2-amplitude:  3.39e+02
reduced chi2:  410.39678874188
******************

***************************************************
Z2-amplitude:  3.58e+02
reduced chi2:  300.8768958704935
***************************************************
Z2-amplitude:  3.44e+02
reduced chi2:  213.9824493005035
***************************************************
Z2-amplitude:  3.55e+02
reduced chi2:  291.3755447605301
***************************************************
Z2-amplitude:  3.65e+02
reduced chi2:  288.88319039592966
***************************************************
Z2-amplitude:  3.67e+02
reduced chi2:  450.0059527955875
***************************************************
Z2-amplitude:  3.47e+02
reduced chi2:  316.25281054497447
***************************************************
Z2-amplitude:  3.51e+02
reduced chi2:  326.8934319739898
***************************************************
Z2-amplitude:  3.41e+02
reduced chi2:  293.52371067339936
***************************************************
Z2-amplitude:  3.45e+02
reduced chi2:  252.20218890801158
***************

***************************************************
Z2-amplitude:  3.43e+02
reduced chi2:  268.69287909980096
***************************************************
Z2-amplitude:  3.42e+02
reduced chi2:  254.7663285589757
***************************************************
Z2-amplitude:  3.49e+02
reduced chi2:  370.24935926707457
***************************************************
Z2-amplitude:  3.63e+02
reduced chi2:  353.3705310296242
***************************************************
Z2-amplitude:  3.53e+02
reduced chi2:  304.89908918654584
***************************************************
Z2-amplitude:  3.41e+02
reduced chi2:  222.5519937228857
***************************************************
Z2-amplitude:  3.49e+02
reduced chi2:  416.35762620311345
***************************************************
Z2-amplitude:  3.54e+02
reduced chi2:  242.15284861232277
***************************************************
Z2-amplitude:  3.51e+02
reduced chi2:  320.73862429991317
*************

***************************************************
Z2-amplitude:  3.59e+02
reduced chi2:  267.11740726313224
***************************************************
Z2-amplitude:  3.48e+02
reduced chi2:  268.3517458725098
***************************************************
Z2-amplitude:  3.57e+02
reduced chi2:  453.1451730221318
***************************************************
Z2-amplitude:  3.49e+02
reduced chi2:  286.4555788043242
***************************************************
Z2-amplitude:  3.60e+02
reduced chi2:  219.66390076056175
***************************************************
Z2-amplitude:  3.64e+02
reduced chi2:  407.1967055916941
***************************************************
Z2-amplitude:  3.57e+02
reduced chi2:  335.4197953855183
***************************************************
Z2-amplitude:  3.51e+02
reduced chi2:  260.6177499258967
***************************************************
Z2-amplitude:  3.52e+02
reduced chi2:  329.2098188529067
*****************

***************************************************
Z2-amplitude:  3.51e+02
reduced chi2:  230.36167704294573
***************************************************
Z2-amplitude:  3.63e+02
reduced chi2:  310.68683580388773
***************************************************
Z2-amplitude:  3.51e+02
reduced chi2:  356.49912692215634
***************************************************
Z2-amplitude:  3.66e+02
reduced chi2:  346.8701644357554
***************************************************
Z2-amplitude:  3.61e+02
reduced chi2:  376.8596538919661
***************************************************
Z2-amplitude:  3.63e+02
reduced chi2:  342.63674231838246
***************************************************
Z2-amplitude:  3.49e+02
reduced chi2:  236.16457957320895
***************************************************
Z2-amplitude:  3.46e+02
reduced chi2:  373.5571590212556
***************************************************
Z2-amplitude:  3.48e+02
reduced chi2:  444.20075767183727
*************

...

***************************************************
Z2-amplitude:  3.40e+02
reduced chi2:  265.48853282727845
***************************************************
Z2-amplitude:  3.50e+02
reduced chi2:  215.14091902104423
***************************************************
Z2-amplitude:  3.45e+02
reduced chi2:  328.08346184068984
***************************************************
Z2-amplitude:  3.42e+02
reduced chi2:  266.19178054895787
***************************************************
Z2-amplitude:  3.41e+02
reduced chi2:  310.5222017865241
***************************************************
Z2-amplitude:  3.37e+02
reduced chi2:  386.2748478374623
***************************************************
Z2-amplitude:  3.28e+02
reduced chi2:  297.846261271082
***************************************************
Z2-amplitude:  3.29e+02
reduced chi2:  229.77370867243778
***************************************************
Z2-amplitude:  3.41e+02
reduced chi2:  318.03697208127704
**************

***************************************************
Z2-amplitude:  3.44e+02
reduced chi2:  288.52061877060305
***************************************************
Z2-amplitude:  3.40e+02
reduced chi2:  211.948616975848
***************************************************
Z2-amplitude:  3.34e+02
reduced chi2:  221.74162265013175
***************************************************
Z2-amplitude:  3.39e+02
reduced chi2:  252.34982609643984
***************************************************
Z2-amplitude:  3.37e+02
reduced chi2:  191.62146936343504
***************************************************
Z2-amplitude:  3.48e+02
reduced chi2:  305.3873454489536
***************************************************
Z2-amplitude:  3.50e+02
reduced chi2:  307.25271849867266
***************************************************
Z2-amplitude:  3.45e+02
reduced chi2:  300.85691362496317
***************************************************
Z2-amplitude:  3.32e+02
reduced chi2:  248.09196716695573
*************

***************************************************
Z2-amplitude:  3.51e+02
reduced chi2:  259.98517795195454
***************************************************
Z2-amplitude:  3.50e+02
reduced chi2:  210.669366261751
***************************************************
Z2-amplitude:  3.55e+02
reduced chi2:  300.1160130248955
***************************************************
Z2-amplitude:  3.50e+02
reduced chi2:  571.1078803243162
***************************************************
Z2-amplitude:  3.49e+02
reduced chi2:  219.16060331691637
***************************************************
Z2-amplitude:  3.55e+02
reduced chi2:  268.2753775471728
***************************************************
Z2-amplitude:  3.55e+02
reduced chi2:  365.19892077754446
***************************************************
Z2-amplitude:  3.49e+02
reduced chi2:  306.2156061868905
***************************************************
Z2-amplitude:  3.45e+02
reduced chi2:  253.22237012312502
****************

***************************************************
Z2-amplitude:  3.40e+02
reduced chi2:  265.57954612347044
***************************************************
Z2-amplitude:  3.50e+02
reduced chi2:  416.2388786750087
***************************************************
Z2-amplitude:  3.39e+02
reduced chi2:  303.6420976911568
***************************************************
Z2-amplitude:  3.46e+02
reduced chi2:  205.77127415918355
***************************************************
Z2-amplitude:  3.46e+02
reduced chi2:  261.2807489744565
***************************************************
Z2-amplitude:  3.31e+02
reduced chi2:  236.99525860118936
***************************************************
Z2-amplitude:  3.37e+02
reduced chi2:  211.06857519427712
***************************************************
Z2-amplitude:  3.28e+02
reduced chi2:  239.28890824228026
***************************************************
Z2-amplitude:  3.26e+02
reduced chi2:  171.068313538991
***************

***************************************************
Z2-amplitude:  3.36e+02
reduced chi2:  287.2921268854883
***************************************************
Z2-amplitude:  3.39e+02
reduced chi2:  202.38161643548207
***************************************************
Z2-amplitude:  3.36e+02
reduced chi2:  346.422965428325
***************************************************
Z2-amplitude:  3.29e+02
reduced chi2:  278.91457014404057
***************************************************
Z2-amplitude:  3.37e+02
reduced chi2:  299.79946707790583
***************************************************
Z2-amplitude:  3.35e+02
reduced chi2:  316.52973862489284
***************************************************
Z2-amplitude:  3.25e+02
reduced chi2:  277.89606482750804
***************************************************
Z2-amplitude:  3.32e+02
reduced chi2:  218.98208790698627
***************************************************
Z2-amplitude:  3.37e+02
reduced chi2:  193.52983303192644
*************

In [237]:
amps = [m1_.values[0] for m1_ in m1_zscale]

In [239]:
from scipy import signal

In [245]:
_,ax = plt.subplots(figsize=(9.2,4))
amps_det = signal.detrend(amps)
ax.scatter(range(len(amps_det)), amps_det)
np.std(amps_det)/np.mean(amps)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

-0.026077363451611223